In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import pi
import cv2
import scipy.misc
import tensorflow as tf

In [2]:
DATA_FOLDER = "D:\\[FCO] AppliedAICourse - Applied Machine Learning Course\\44. Self Driving Cars RWCS\\New folder\\Autopilot-TensorFlow-master\\driving_dataset\\"
DATA_FILE = os.path.join(DATA_FOLDER, "data.txt")

In [3]:
x = []
y = []

train_batch_pointer = 0
test_batch_pointer = 0

In [4]:
with open(DATA_FILE) as f:
    for line in f:
        image_name, angle = line.split()
        
        image_path = os.path.join(DATA_FOLDER, image_name)
        x.append(image_path)
        
        angle_radians = float(angle) * (pi / 180)  #converting angle into radians
        y.append(angle_radians)
y = np.array(y)

In [5]:
split_ratio = int(len(x) * 0.8)

train_x = x[:split_ratio]
train_y = y[:split_ratio]

test_x = x[split_ratio:]
test_y = y[split_ratio:]

In [6]:
def weightVariable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def convolution(previous_input, filter_input, strides):
    return tf.nn.conv2d(previous_input, filter_input, strides = [1, strides, strides, 1], padding = "VALID")

In [7]:
x_input = tf.placeholder(tf.float32, shape = [None, 66, 200, 3], name = "Plc_1")
y_true = tf.placeholder(tf.float32, name = "Plc_2")

In [8]:
input_image = x_input

In [9]:
#Convolution Layers
#First convolution layer
W_Conv1 = weightVariable([5,5,3,24])
B_Conv1 = bias_variable([24])
Conv1 = tf.nn.relu(convolution(input_image, W_Conv1, 2) + B_Conv1)
#strides = 2
#Output size: 31*98*24

In [10]:
#Second convolution layer
W_Conv2 = weightVariable([5,5,24,36])
B_Conv2 = bias_variable([36])
Conv2 = tf.nn.relu(convolution(Conv1, W_Conv2, 2) + B_Conv2)
#strides = 2
#Output size: 14*47*36

In [11]:
#Third convolution layer
W_Conv3 = weightVariable([5,5,36,48])
B_Conv3 = bias_variable([48])
Conv3 = tf.nn.relu(convolution(Conv2, W_Conv3, 2) + B_Conv3)
#strides = 2
#Output size: 5*22*48

In [12]:
#Fourth convolution layer
W_Conv4 = weightVariable([3,3,48,64])
B_Conv4 = bias_variable([64])
Conv4 = tf.nn.relu(convolution(Conv3, W_Conv4, 1) + B_Conv4)
#strides = 1
#Output size: 3*20*64

In [13]:
#Fifth convolution layer
W_Conv5 = weightVariable([3,3,64,64])
B_Conv5 = bias_variable([64])
Conv5 = tf.nn.relu(convolution(Conv4, W_Conv5, 1) + B_Conv5)
#strides = 1
#Output size: 1*18*64

In [14]:
#Fully-Connected Dense Layers
keep_prob = tf.placeholder(tf.float32)
#First FC-Dense
#Input = 1*18*64 = 1152
W_FC1 = weightVariable([1152, 1164])
B_FC1 = bias_variable([1164])
FC1_Flatten = tf.reshape(Conv5, [-1, 1152]) #here, -1 indicates 1. It means that the shape of FC1_Flatten will be 1*1152
Output_FC1 = tf.nn.relu(tf.matmul(FC1_Flatten, W_FC1) + B_FC1) #so, here shape of FC1_Flatten is 1*1152 and shape of W_FC1 will
#be 1152*1164. Therefore, there will be a matrix multiplication of matrices: (1*1152) * (1152*1164) = (1*1164).
Output_FC1_drop = tf.nn.dropout(Output_FC1, keep_prob)

In [15]:
#Second FC-Dense
#Input = 1*1164 = 1164
W_FC2 = weightVariable([1164, 100])
B_FC2 = bias_variable([100])
Output_FC2 = tf.nn.relu(tf.matmul(Output_FC1_drop, W_FC2) + B_FC2) #so, here shape of Output_FC1_drop is 1*1164 and shape of 
#W_FC2 will be 1164*100. Therefore, there will be a matrix multiplication of matrices: (1*1164) * (1164*100) = (1*100).
Output_FC2_drop = tf.nn.dropout(Output_FC2, keep_prob)

In [16]:
#Third FC-Dense
#Input = 1*100 = 100
W_FC3 = weightVariable([100, 50])
B_FC3 = bias_variable([50])
Output_FC3 = tf.nn.relu(tf.matmul(Output_FC2_drop, W_FC3) + B_FC3) #so, here shape of Output_FC2_drop is 1*100 and shape of 
#W_FC3 will be 100*50. Therefore, there will be a matrix multiplication of matrices: (1*100) * (100*50) = (1*50).
Output_FC3_drop = tf.nn.dropout(Output_FC3, keep_prob)

In [17]:
#Fourth FC-Dense
#Input = 1*50 = 50
W_FC4 = weightVariable([50, 10])
B_FC4 = bias_variable([10])
Output_FC4 = tf.nn.relu(tf.matmul(Output_FC3_drop, W_FC4) + B_FC4) #so, here shape of Output_FC3_drop is 1*50 and shape of 
#W_FC4 will be 50*10. Therefore, there will be a matrix multiplication of matrices: (1*50) * (50*10) = (1*10).
Output_FC4_drop = tf.nn.dropout(Output_FC4, keep_prob)

In [18]:
#Final Output to one neuron with linear/identity function
#Input = 1*10 = 10
W_FC5 = weightVariable([10, 1])
B_FC5 = bias_variable([1])
y_predicted = tf.identity(tf.matmul(Output_FC4_drop, W_FC5) + B_FC5)

In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [20]:
sess = tf.InteractiveSession(config=config)
saver = tf.train.Saver()
saver.restore(sess, "D:/[FCO] AppliedAICourse - Applied Machine Learning Course/44. Self Driving Cars RWCS/New folder/Autopilot-TensorFlow-master/Saver/model.ckpt")

INFO:tensorflow:Restoring parameters from D:/[FCO] AppliedAICourse - Applied Machine Learning Course/44. Self Driving Cars RWCS/New folder/Autopilot-TensorFlow-master/Saver/model.ckpt


In [21]:
img = cv2.imread('D:\\[FCO] AppliedAICourse - Applied Machine Learning Course\\44. Self Driving Cars RWCS\\New folder\\Autopilot-TensorFlow-master\\Steering-Wheel-Nissan-GT-R-Wallpaper-Free1.jpg', 0)
rows, cols = img.shape

In [22]:
i = 0

In [23]:
while(cv2.waitKey(50) != ord("q")):
    full_image = cv2.imread(test_x[i])
    cv2.imshow('Frame Window', full_image)
    image = ((cv2.resize(full_image[-150:], (200, 66)) / 255.0).reshape((1, 66, 200, 3)))
    degrees = sess.run(y_predicted, feed_dict = {x_input: image, keep_prob: 1.0})[0][0] *180 / pi
    print("Predicted degrees: "+str(degrees))
    M = cv2.getRotationMatrix2D((cols/2,rows/2), -degrees, 1)
    dst = cv2.warpAffine(src = img, M = M, dsize = (cols, rows))
    cv2.imshow("Steering Wheel", dst)
    i += 1

cv2.destroyAllWindows()

Predicted degrees: -0.9526875920380723
Predicted degrees: 15.341897489149064
Predicted degrees: 16.88665576052146
Predicted degrees: 15.16009492891625
Predicted degrees: 24.26268803579224
Predicted degrees: 21.649956263154774
Predicted degrees: 16.951926755777393
Predicted degrees: 20.670647155052933
Predicted degrees: 18.536199379045605
Predicted degrees: 15.426074448027594
Predicted degrees: 12.556616355056857
Predicted degrees: 11.496719552485326
Predicted degrees: 10.079600441227543
Predicted degrees: 6.836936543139077
Predicted degrees: 7.178172794072914
Predicted degrees: 4.854369976158034
Predicted degrees: 4.337821552250187
Predicted degrees: 3.9414622596197617
Predicted degrees: 5.3886098706766825
Predicted degrees: 5.744989931660908
Predicted degrees: 5.345509242579787
Predicted degrees: 1.602269744844373
Predicted degrees: 5.892706859727596
Predicted degrees: 5.981725995840842
Predicted degrees: 5.066350873464903
Predicted degrees: 7.4310494490352355
Predicted degrees: 6.947